# Esercizio 2: Automatic Summarization

- L'obiettivo del text summarization è quello di produrre una versione ridotta di un testo che contenga le informazioni importanti o pertinenti.
    - un riassunto di un articolo scientifico, un riepilogo dei thread di posta elettronica, un titolo per un articolo di notizie o i brevi frammenti restituiti dai motori di ricerca web per descrivere ogni documento recuperato.

In [18]:
import re

In [19]:
ddsmallnasari = {}

"""
Forma del dizionario:

{'million': ['bn:00000013n',
  'million',
  ...,
  'infinity'],
 'day': ['bn:00000086n',
  'day',
  ...,
  'time',
  'clock']
}

"""

def initialize_ddsmallnasari():
    with open(f'NASARI/dd-small-nasari-15.txt', 'r', encoding="utf8") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.split(";")
        key = parts[1].lower()
        var = []
        for p in parts:
            u = p.split("_")
            var.append(u[0].lower())
            
        ddsmallnasari[key] = var

In [26]:
def read_doc(doc):
    document = []
    temp_document = []
    with open('docs/{}'.format(doc), 'r', encoding="utf8") as f:
        lines = f.readlines()
    for line in lines:
        if(line!="\n"):
            temp_document.append(line)

    for elem in temp_document:
        cleanString = re.sub(r"[^a-zA-Z0-9]+", ' ', elem)
        document.append(cleanString)

    return document

In [29]:
docs = ["Andy-Warhol.txt", "Ebola-virus-disease.txt", "Life-indoors.txt", "Napoleon-wiki.txt", "Trump-wall.txt"]

document = read_doc(docs[3])

document

['Napoleone Bonaparte ',
 'Napoleon Bonaparte born Napoleone di Buonaparte was a French statesman and military leader who rose to prominence during the French Revolution and led several successful campaigns during the French Revolutionary Wars ',
 'He was Emperor of the French as Napoleon I from 1804 until 1814 and again briefly in 1815 during the Hundred Days Napoleon dominated European and global affairs for more than a decade while leading France against a series of coalitions in the Napoleonic Wars ',
 'He won most of these wars and the vast majority of his battles building a large empire that ruled over much of continental Europe before its final collapse in 1815 He is considered one of the greatest commanders in history and his wars and campaigns are studied at military schools worldwide ',
 'Napoleon s political and cultural legacy has endured as one of the most celebrated and controversial leaders in human history ',
 'He was born in Corsica to a relatively modest Italian famil

### Funzione che calcola la Weighted Overlap

In [ ]:
def WO(v1,v2): # Weighted Overlap
    numeratore = 0
    denominatore = 0
    if v1 in ddsmallnasari and v2 in ddsmallnasari:
        vector1 = ddsmallnasari[v1].copy()
        vector2 = ddsmallnasari[v2].copy()

        O = set(vector1).intersection(set(vector2)) # insieme di dimensioni sovrapposte

        # calcolo numeratore
        for q in O:
            rankv1 = vector1.index(q) + 1 # + 1 per evitare di poter avere 0 al denominatore
            rankv2 = vector2.index(q) + 1 # + 1 per evitare di poter avere 0 al denominatore
            numeratore += 1/(rankv1 + rankv2)
        
        # calcolo denominatore
        for i in range(0, len(O)):
            denominatore += 1/(2*i)
        
    if denominatore != 0:
        return numeratore/denominatore
    else:
        return 0
    